
This notebook performs maximum likelihood estimation to find parameters in a function that returns the probability of COVID-19 transmission given distance over a two-dimensional space. 

1D droplet deposition model: Sun and Zhai 2020 https://doi.org/10.1016/j.scs.2020.102390

Exponential dose response model: Buonanno et al. 2020 https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7474922/ 

Train data: Hu et al. 2021 https://doi.org/10.1093/cid/ciaa1057

# Approach: Maximum Likelihood Estimation


The fraction of close contacts seated $x$ rows and $y$ columns away from an index case that are in the $\alpha$ cone of exposure is
$$
q_\alpha(x,y) = \frac{1}{2} + \frac{1}{2} \mathbf{1}\{\arctan\left({d_r(x,y)}/{d_c(x,y)}\right) \leq \alpha\},
$$
where $d_r$ and $d_c$ denote the row-wise and column-wise distance between the pair.

The probability of transmission for a close contact at $(x,y)$ within the cone of exposure is
$$p_{c_2}(d(x,y)) = 1-\exp\left(-c_2\cdot \frac{-0.1819\cdot \ln d(x,y) + 0.43276}{d(x,y)}\cdot T\right),$$
where $T$ is the mean co-travel time.

We assume $Y(x,y)\sim \text{Binomial} (N(x,y), q_\alpha(x,y)p_{c_2}(d(x,y)) )$. 

Denote $\nu = (\alpha, c_2)$.

The likelihood function:
$$L(\nu) = \prod_{(x,y)} \binom{N(x,y)}{Y(x,y)}\cdot\left(q_\alpha(x,y)p_{c_2}(d(x,y))\right)^{Y(x,y)}\cdot \left(1-q_\alpha(x,y)p_{c_2}(d(x,y))\right)^{N(x,y)-Y(x,y)}$$


The log likelihood function:
$\log L(\nu) = \sum_{(x,y)} \log\binom{N(x,y)}{Y(x,y)}+\left[ Y(x,y)\log\left(q_\alpha(x,y)p_{c_2}(d(x,y))\right) + (N(x,y)-Y(x,y))\log\left(1-q_\alpha(x,y)p_{c_2}(d(x,y))\right)\right]$


We want to find the maximum likelihood estimator:
$$(\alpha^*, c_2^*) = \nu^* = \arg\max_\nu \log L(\nu).$$


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.special import binom

In [60]:
# data from high-speed train

unit_distance = 0.5
mean_cotravel_time = 2.1

# number of close contacts at seats (x rows, y cols) apart from an index case
attack_rates_counts = np.array([[1, 2605, 1996, 1845, 1825, 1028],
                             [4791, 5084, 3664, 3464, 3525, 1872],
                             [4386, 4751, 3429, 3212, 3250, 1769],
                             [4026, 4395, 3110, 2945, 2970, 1589]])
counts = attack_rates_counts.flatten()[1:]

# number of close contacts at seats (x rows, y cols) apart from an index case
# that were later confirmed as positive
positive_counts = np.array([[0, 92, 33, 7, 7, 3],
                            [10, 12, 5, 3, 1, 1],
                            [11, 8, 8, 5, 3, 3],
                            [2, 2, 4, 3, 3, 1]])
num_pos_vec = positive_counts.flatten()[1:]

attack_rates = np.array([[100, 3.53, 1.65, 0.38, 0.38, 0.29],
                         [0.21, 0.24, 0.14, 0.09, 0.03, 0.05],
                         [0.25, 0.17, 0.23, 0.16, 0.09, 0.17],
                         [0.05, 0.05, 0.13, 0.10, 0.10, 0.06]]) * 0.01

row_distance = 0.9 * np.arange(4)
col_distance = np.array([0, 0.5, 1.05, 1.6, 2.1, 2.6])

distance_matrix = np.array([[np.sqrt(row_distance[i]**2 + col_distance[j]**2) for j in range(6)] for i in range(4)])
distance_vec = distance_matrix.flatten()[1:]

# fraction of droplets that travel beyond distance d
compute_beta = lambda d: (-0.1819*np.log(d)+0.43276)/d if d > 0.044 else 1

gamma_mask = 0.8

In [42]:
print(distance_matrix)

[[0.         0.5        1.05       1.6        2.1        2.6       ]
 [0.9        1.02956301 1.38293167 1.83575598 2.28473193 2.7513633 ]
 [1.8        1.86815417 2.0838666  2.40831892 2.76586334 3.16227766]
 [2.7        2.74590604 2.89698119 3.13847097 3.42052628 3.74833296]]


In [43]:
np.sum(attack_rates_counts)

71532

In [44]:
# Computes the angle that the susceptible is located wrt the source (horizontal neighbor = zero)
angle_matrix = np.array([[np.arctan(row_distance[i]/col_distance[j])*180/np.pi for j in range(6)] for i in range(4)])
angle_matrix[0,0]=0

/tmp/ipykernel_814812/1016738103.py:2: RuntimeWarning: invalid value encountered in double_scalars
  angle_matrix = np.array([[np.arctan(row_distance[i]/col_distance[j])*180/np.pi for j in range(6)] for i in range(4)])
/tmp/ipykernel_814812/1016738103.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  angle_matrix = np.array([[np.arctan(row_distance[i]/col_distance[j])*180/np.pi for j in range(6)] for i in range(4)])


In [45]:
angle_matrix

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [90.        , 60.9453959 , 40.60129465, 29.35775354, 23.19859051,
        19.093492  ],
       [90.        , 74.475889  , 59.74356284, 48.36646066, 40.60129465,
        34.69515353],
       [90.        , 79.50852299, 68.74949449, 59.34933204, 52.12501635,
        46.08092419]])

In [61]:
# compute the fraction of people counted at (|row|, |col|) that are seated within the cone of exposure
# extending from (-threshold_angle, 180 + threshold_angle) degrees

# some seats at (-|row|, +-|col|), i.e., behind the index case, are not in the cone of exposure

def fraction_in_cone(threshold_angle, scale_factor = 0.5, angle_matrix = angle_matrix):
  frac = np.zeros((4,6))
  for i in range(4):
    for j in range(6):
      if angle_matrix[i,j] < threshold_angle:
        frac[i,j]=1
      else:
        frac[i,j]=scale_factor
    
  return frac

In [62]:
# As an example, we compute the fraction_in_cone_matrix for a threshold angle
# the output matrix represents ... (TODO: explain; make consistent with doc)
critical_angle = 60

frac_in_cone_matrix = fraction_in_cone(60)
print(frac_in_cone_matrix)

[[1.  1.  1.  1.  1.  1. ]
 [0.5 0.5 1.  1.  1.  1. ]
 [0.5 0.5 1.  1.  1.  1. ]
 [0.5 0.5 0.5 1.  1.  1. ]]


In [63]:
# define function to compute 
# the fraction of droplets received at distance d (called phi(d) in the paper) divide by d
# define beta(d) = phi(d) / d

def compute_beta(d, lambdaa=1):
  if d > 0.044:
    return (-0.1819*np.log(d)+0.43276)/(d**lambdaa)
  else:
    return 1

In [65]:
beta_matrix = np.array([[compute_beta(distance_matrix[i,j]) for j in range(6)] for i in range(4)])

In [18]:
def compute_log_likelihood(angle, constant, lambdaa, N=counts, Y=num_pos_vec, d=distance_vec, gamma_mask = gamma_mask):
  # compute frac q for given angle
  fraction_in_cone_matrix = fraction_in_cone(angle)
  q = fraction_in_cone_matrix.flatten()[1:]
  
  # compute log likelihood for given q and constant
  log_llh = 0
  for i in range(len(counts)):
    prob_trans = q[i] * (1-np.exp(-constant * mean_cotravel_time * gamma_mask * compute_beta(d[i],lambdaa)))
    log_llh += np.log(binom(N[i], Y[i])) + (Y[i] * np.log(prob_trans) + (N[i]-Y[i])*np.log(1-prob_trans))
  
  return log_llh

In [51]:
# lambdaa_grid = np.linspace(1,2,101)
angle_grid = np.linspace(0, 30, 31)
const_grid = np.linspace(0, 0.03, 61)

In [53]:
log_llh_array = np.zeros((len(angle_grid), len(const_grid)))

for angle_idx in range(len(angle_grid)):
  for const_idx in range(len(const_grid)):
    log_llh_array[angle_idx, const_idx] = compute_log_likelihood(angle_grid[angle_idx], const_grid[const_idx], lambdaa = 1)

/tmp/ipykernel_814812/1011737979.py:10: RuntimeWarning: divide by zero encountered in log
  log_llh += np.log(binom(N[i], Y[i])) + (Y[i] * np.log(prob_trans) + (N[i]-Y[i])*np.log(1-prob_trans))


In [54]:
# find the values of (alpha, constant) that maximize the likelihood

max_alpha_idx, max_c_idx = np.where(log_llh_array == np.amax(log_llh_array))
print(max_alpha_idx, max_c_idx)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] [27 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27]


In [55]:
angle_grid[max_alpha_idx]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19.])

In [56]:
const_grid[max_c_idx]

array([0.0135, 0.0135, 0.0135, 0.0135, 0.0135, 0.0135, 0.0135, 0.0135,
       0.0135, 0.0135, 0.0135, 0.0135, 0.0135, 0.0135, 0.0135, 0.0135,
       0.0135, 0.0135, 0.0135])

## Conclusion: 

maximizer of likelihood is alpha = 19 degrees, constant = 0.0135

In [57]:
fraction_in_cone(angle_grid[1])

array([[1. , 1. , 1. , 1. , 1. , 1. ],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]])